# Accidentally Deleted all Renditions

Maybe I can regenerate all of them in a notebook.

In [26]:
def regenerate_renditions_for_image(image):
    new_renditions = []
    for rendition in image.renditions.all():
        r = httpx.head(rendition.url)
        if r.status_code == 200:
            continue
        # create new rendition - don't use get_rendition, because it will not create a file if
        # a rendition with the same filter already exists
        rendition_filter = rendition.filter
        rendition.delete()
        new_renditions.append(image.create_rendition(rendition.filter))
    return new_renditions
        


def regenerate_renditions_for_post(post):
    new_renditions = []
    for image in post.images.all():
        new_renditions.extend(regenerate_renditions_for_image(image))
    for gallery in post.galleries.all():
        for image in gallery.images.all():
            new_renditions.extend(regenerate_renditions_for_image(image))
    return new_renditions

In [27]:
%%time
post = Post.objects.get(slug="weeknotes-2023-07-31")

CPU times: user 2.09 ms, sys: 967 µs, total: 3.06 ms
Wall time: 2.19 ms


In [28]:
%%time
new_renditions = regenerate_renditions_for_post(post)

CPU times: user 1min 52s, sys: 22.9 s, total: 2min 15s
Wall time: 4min 22s


In [12]:
rendition = new_renditions[0]
print(rendition)

Rendition object (10047)


In [20]:
rendition.url

'https://d2b7dn9rofvhjd.cloudfront.net/images/IMG_0738.max-800x600.jpg'

In [22]:
rendition_filter = rendition.filter
image = rendition.image
rendition.delete()
new_rendition = image.create_rendition(rendition_filter)

In [23]:
new_rendition.url

'https://d2b7dn9rofvhjd.cloudfront.net/images/IMG_0738.max-800x600.jpg'

In [10]:
image = rendition.image

In [11]:
image.find_existing_rendition(rendition.filter)

<Rendition: Rendition object (10047)>

In [13]:
import httpx

In [14]:
r = httpx.head(rendition.url)

In [15]:
r.status_code

403

In [18]:
rendition_by_status = {}
for num, rendition in enumerate(Rendition.objects.all()):
    r = httpx.head(rendition.url)
    rendition_by_status.setdefault(r.status_code, []).append(rendition)
    if num % 100 == 0:
        print("------------------------------")
        print({k: len(v) for k, v in rendition_by_status.items()})

------------------------------
{403: 1}
------------------------------
{403: 101}
------------------------------
{403: 201}
------------------------------
{403: 301}
------------------------------
{403: 401}
------------------------------
{403: 501}
------------------------------
{403: 601}
------------------------------
{403: 701}
------------------------------
{403: 801}
------------------------------
{403: 901}
------------------------------
{403: 1001}
------------------------------
{403: 1101}
------------------------------
{403: 1201}
------------------------------
{403: 1301}
------------------------------
{403: 1401}
------------------------------
{403: 1501}
------------------------------
{403: 1601}
------------------------------
{403: 1701}
------------------------------
{403: 1801}
------------------------------
{403: 1901}
------------------------------
{403: 2001}
------------------------------
{403: 2101}
------------------------------
{403: 2201}
-----------------------

In [29]:
blog = Blog.objects.get(slug="ephes_blog")

In [30]:
Post.objects.all().descendant_of(blog).order_by("-visible_date")

<PageQuerySet [<Post: Weeknotes 2023-09-11>, <Post: Weeknotes 2023-09-04>, <Post: Weeknotes 2023-08-28>, <Post: Weeknotes 2023-08-21>, <Post: Not Getting Payed>, <Post: Weeknotes 2023-08-14>, <Post: Testing Django Management Commands with PyTest>, <Post: TIL: Using staticfiles with STORAGES>, <Post: Estimates Considered Harmful>, <Post: Weeknotes 2023-08-07>, <Post: Weeknotes 2023-07-31>, <Post: Weeknotes 2023-07-24>, <Post: Weeknotes 2023-07-17>, <Post: Weeknotes 2023-07-10>, <Post: Weeknotes 2023-07-03>, <Post: Weeknotes 2023-06-26>, <Post: Weeknotes 2023-06-19>, <Post: Weeknotes 2023-06-12>, <Post: Weeknotes 2023-06-05>, <Post: Weeknotes 2023-05-29>, '...(remaining elements truncated)...']>